# Method-I

In [61]:
import math
import numpy as np
import matplotlib.pyplot as plt
plt.ioff() # interactive mode off
# from matplotlib.patches import Rectangle
from matplotlib import animation
import pandas as pd
# plt.rcParams['figure.figsize']=(16,9)
plt.rcParams.update({'font.size': 13})
plt.rcParams['animation.html'] = 'html5'
# plt.rcParams['text.usetex'] = True
# %config InlineBackend.close_figures=False # keep figures open in pyplot

In [62]:
def postproc(x1, x2, x3, T=10, N=100, P=[1,1,1], E=2000000, nu=0.2, limit=[-12,12,-12,12], animx=1):
    """
    X1,X2,X3 - material coordinates
    x1,x2,x3 - spacial coordinates
    t - current time(sec)
    T - total time(sec)
    N - no of time steps
    P - point at which the F,C,b,E,e,sigma,S are evaluated
    """
    ## Shape Functions and Their derivatives w.r.to X1,X2,X3 -- Linear Hexahedral Element
    # ---------------------------------------------------------- Shape Functions
    def N1(X1,X2,X3): return 0.125*(1-X1)*(1+X2)*(1+X3)
    def N2(X1,X2,X3): return 0.125*(1+X1)*(1+X2)*(1+X3)
    def N3(X1,X2,X3): return 0.125*(1-X1)*(1-X2)*(1+X3)
    def N4(X1,X2,X3): return 0.125*(1+X1)*(1-X2)*(1+X3)
    def N5(X1,X2,X3): return 0.125*(1-X1)*(1+X2)*(1-X3)
    def N6(X1,X2,X3): return 0.125*(1+X1)*(1+X2)*(1-X3)
    def N7(X1,X2,X3): return 0.125*(1-X1)*(1-X2)*(1-X3)
    def N8(X1,X2,X3): return 0.125*(1+X1)*(1-X2)*(1-X3)
    # ---------------------------- Derivatives of the shape functions w.r.to 'X1'
    def N1X1(X2,X3): return -0.125*(1+X2)*(1+X3)
    def N2X1(X2,X3): return  0.125*(1+X2)*(1+X3)
    def N3X1(X2,X3): return -0.125*(1-X2)*(1+X3)
    def N4X1(X2,X3): return  0.125*(1-X2)*(1+X3)
    def N5X1(X2,X3): return -0.125*(1+X2)*(1-X3)
    def N6X1(X2,X3): return  0.125*(1+X2)*(1-X3)
    def N7X1(X2,X3): return -0.125*(1-X2)*(1-X3)
    def N8X1(X2,X3): return  0.125*(1-X2)*(1-X3)
    # ---------------------------- Derivatives of the shape functions w.r.to 'X2'
    def N1X2(X1,X3): return  0.125*(1-X1)*(1+X3)
    def N2X2(X1,X3): return  0.125*(1+X1)*(1+X3)
    def N3X2(X1,X3): return -0.125*(1-X1)*(1+X3)
    def N4X2(X1,X3): return -0.125*(1+X1)*(1+X3)
    def N5X2(X1,X3): return  0.125*(1-X1)*(1-X3)
    def N6X2(X1,X3): return  0.125*(1+X1)*(1-X3)
    def N7X2(X1,X3): return -0.125*(1-X1)*(1-X3)
    def N8X2(X1,X3): return -0.125*(1+X1)*(1-X3)
    # ---------------------------- Derivatives of the shape functions w.r.to 'X3'
    def N1X3(X1,X2): return  0.125*(1-X1)*(1+X2)
    def N2X3(X1,X2): return  0.125*(1+X1)*(1+X2)
    def N3X3(X1,X2): return  0.125*(1-X1)*(1-X2)
    def N4X3(X1,X2): return  0.125*(1+X1)*(1-X2)
    def N5X3(X1,X2): return -0.125*(1-X1)*(1+X2)
    def N6X3(X1,X2): return -0.125*(1+X1)*(1+X2)
    def N7X3(X1,X2): return -0.125*(1-X1)*(1-X2)
    def N8X3(X1,X2): return -0.125*(1+X1)*(1-X2)

    def eqstress(S):
        Sd = S - (S.trace()/3)*I
        Seq = math.sqrt(1.5*np.tensordot(Sd,Sd))
        return Seq
    
    def eqstrain(E,nu):
        dummy = ((E[0,0]-E[1,1])**2 + E[1,1]-E[2,2])**2 + (E[2,2]-E[0,0])**2 + 3*(E[0,1]**2+E[1,2]**2+E[2,0]**2)
        Eeq = (1/(math.sqrt(2)*(1+nu)))*math.sqrt(dummy)
        return Eeq

    dt = T/N
    lmda = (E*nu)/((1+nu)*(1-2*nu))
    mu   = 1/(2*(1+nu))
    I = np.array([[1,0,0],[0,1,0],[0,0,1]])

    nodes_t0 = np.array([[-1,1,1],[1,1,1],[-1,-1,1],[1,-1,1],[-1,1,-1],[1,1,-1],[-1,-1,-1],[1,-1,-1]])
    xnodes_ndt = np.zeros((N+1,8,3))
    xnodes_ndt[0,:,:] = nodes_t0
    unodes_ndt = np.zeros((N+1,8,3))
    
    point_ndt = np.zeros((N+1,3))
    point_ndt[0,:] = P
    grad_u_P = np.zeros((N+1,3,3))
    F_P = np.zeros((N+1,3,3))
    C_P = np.zeros((N+1,3,3))
    b_P = np.zeros((N+1,3,3))
    E_P = np.zeros((N+1,3,3))
    e_P = np.zeros((N+1,3,3))
    S_P = np.zeros((N+1,3,3))
    sigma_P = np.zeros((N+1,3,3))
    Eeq_P = np.zeros((N+1))
    eeq_P = np.zeros((N+1))
    Seq_P = np.zeros((N+1))
    sigmaeq_P = np.zeros((N+1))


    for n in range(N):
        t = n*dt
        # Calculation of nodal positions and displacements
        for i in range(8):
            X1 = xnodes_ndt[0,i,0]
            X2 = xnodes_ndt[0,i,1]
            X3 = xnodes_ndt[0,i,2]
            xnodes_ndt[n+1,i,0] = eval(x1)
            xnodes_ndt[n+1,i,1] = eval(x2)
            xnodes_ndt[n+1,i,2] = eval(x3)
            unodes_ndt[n+1,i,0] = xnodes_ndt[n+1,i,0] - X1
            unodes_ndt[n+1,i,1] = xnodes_ndt[n+1,i,1] - X2
            unodes_ndt[n+1,i,2] = xnodes_ndt[n+1,i,2] - X3
            
        # Calculation of required quantities at point 'P'
        X1 = P[0]
        X2 = P[1]
        X3 = P[2]
        point_ndt[n+1,0] = eval(x1)
        point_ndt[n+1,1] = eval(x2)
        point_ndt[n+1,2] = eval(x3)
        NX1 = np.array([N1X1(X2,X3), N2X1(X2,X3), N3X1(X2,X3), N4X1(X2,X3), N5X1(X2,X3), N6X1(X2,X3), N7X1(X2,X3), N8X1(X2,X3)])
        NX2 = np.array([N1X2(X1,X3), N2X2(X1,X3), N3X2(X1,X3), N4X2(X1,X3), N5X2(X1,X3), N6X2(X1,X3), N7X2(X1,X3), N8X2(X1,X3)])
        NX3 = np.array([N1X3(X1,X2), N2X3(X1,X2), N3X3(X1,X2), N4X3(X1,X2), N5X3(X1,X2), N6X3(X1,X2), N7X3(X1,X2), N8X3(X1,X2)])
        NX  = np.array([NX1, NX2, NX3])
        for i in range(3):
            for j in range(3):
                grad_u_P[n+1,i,j] = np.dot(unodes_ndt[n+1,:,i],NX[j])
        F_P[n+1,:,:] = I + grad_u_P[n+1,:,:]
        C_P[n+1,:,:] = np.dot(F_P[n+1,:,:].T,F_P[n+1,:,:])
        b_P[n+1,:,:] = np.dot(F_P[n+1,:,:],F_P[n+1,:,:].T)
        E_P[n+1,:,:] = 0.5*(C_P[n+1,:,:] - I)
        e_P[n+1,:,:] = 0.5*(I - np.linalg.inv(b_P[n+1,:,:]))
        J = math.sqrt(np.linalg.det(C_P[n+1,:,:]))
        S_P[n+1,:,:] = mu*(I-np.linalg.inv(C_P[n+1,:,:]))+lmda*math.log(J)*np.linalg.inv(C_P[n+1,:,:])
        sigma_P[n+1,:,:] = (np.dot(F_P[n+1,:,:],np.dot(S_P[n+1,:,:],F_P[n+1,:,:].T)))/(J)
        Eeq_P[n+1] = eqstrain(E_P[n+1,:,:],nu)
        eeq_P[n+1] = eqstrain(e_P[n+1,:,:],nu)
        Seq_P[n+1] = eqstress(S_P[n+1,:,:])
        sigmaeq_P[n+1] = eqstress(sigma_P[n+1,:,:])
        
    # create a figure and axes
    fig = plt.figure(figsize=(16,8), dpi=100)
    dmap = plt.subplot(1,2,1)   
    svs = plt.subplot(1,2,2)

    # set up the subplots as needed
    dmap.set_xlim(limit[0], limit[1])            
    dmap.set_ylim(limit[2], limit[3])
    dmap.axvline(c="black",lw=1,ls='--')
    dmap.axhline(c="black",lw=1,ls='--')
    dmap.set_aspect('equal')
    dmap.set_xlabel('x1')
    dmap.set_ylabel('x2')
    # dmap.set_title('Deformation Map')

    # lxlim = min(round(eeq_P.min()),round(Eeq_P.min()))
    # rxlim = max(round(eeq_P.max()),round(Eeq_P.max()))
    # lylim = min(round(sigmaeq_P.min()),round(Seq_P.min()))
    # rylim = max(round(sigmaeq_P.max()),round(Seq_P.max()))
    lxlim = round(Eeq_P.min())
    rxlim = round(Eeq_P.max())
    lylim = round(Seq_P.min())
    rylim = round(Seq_P.max())
    print(Eeq_P.min(),Eeq_P.max(),eeq_P.min(),eeq_P.max(),Seq_P.min(),Seq_P.max(),sigmaeq_P.min(),sigmaeq_P.max())
    print(lxlim,lylim,rxlim,rylim)
    if lxlim!=0 or rxlim!=0:
        svs.set_xlim(lxlim,rxlim)    
    if lylim!=0 or rylim!=0:
        svs.set_ylim(lylim,rylim)
    svs.axvline(c="black",lw=1,ls='--')
    svs.axhline(c="black",lw=1,ls='--')
    svs.set_box_aspect(1)
    # svs.set_adjustable('datalim')
    # svs.axis('tight')
    svs.set_xlabel(r'$||e||$ or $||E||$')
    svs.set_ylabel(r'$||\sigma||$ or $||S||$')
    # svs.set_xlabel(r'$\|\underline{\underline{e}}\|$ / $\|underline{\underline{E}}\|$', fontsize=16)
    # svs.set_ylabel(r'$\|\underline{\underline{\sigma}}\|$ / $\|\underline{\underline{S}}\|$', fontsize=16)
    svs.set_title('Stress vs Strain')
    

    # create objects that will change in the animation. These are initially empty, and will be given new values for each frame in the animation.
    timestep = dmap.set_title('')
    face5687, = dmap.plot([], [], c="grey", lw=1, ls='--')
    edge15, = dmap.plot([], [], c="grey", lw=1, ls='--')
    edge26, = dmap.plot([], [], c="grey", lw=1, ls='--')
    edge48, = dmap.plot([], [], c="grey", lw=1, ls='--')
    edge37, = dmap.plot([], [], c="grey", lw=1, ls='--')
    face1243, = dmap.plot([], [], 'tab:blue', lw=2.5)
    point, = dmap.plot([], [],'ob', ms=10, label='The point')
    dmap.legend(loc='upper right')

    SvE, = svs.plot([], [], lw=2.5, label=r'$||S||$ vs $||E||$')
    sigmaVe, = svs.plot([], [], lw=2.5, label=r'$||\sigma||$ vs $||e||$')
    svs.legend(loc='upper right')

    # animation function. This is called sequentially
    def drawframe(n, dt=dt, x1x2=xnodes_ndt, S=Seq_P, E=Eeq_P, sigma=sigmaeq_P, e=eeq_P, P=point_ndt):
        face5687.set_data(x1x2[n,(4,5,7,6,4),0], x1x2[n,(4,5,7,6,4),1])
        edge15.set_data(x1x2[n,(0,4),0], x1x2[n,(0,4),1])
        edge26.set_data(x1x2[n,(1,5),0], x1x2[n,(1,5),1])
        edge48.set_data(x1x2[n,(3,7),0], x1x2[n,(3,7),1])
        edge37.set_data(x1x2[n,(2,6),0], x1x2[n,(2,6),1])
        face1243.set_data(x1x2[n,(0,1,3,2,0),0], x1x2[n,(0,1,3,2,0),1])
        point.set_data(point_ndt[n,0],point_ndt[n,1])
        timestep.set_text('Deformed config. at t = {0:2.1f} sec'.format(round(n*dt,1)))

        SvE.set_data(E[:n+1],S[:n+1])
        sigmaVe.set_data(e[:n+1],sigma[:n+1])
        return (face1243,face5687)
    
    anim = animation.FuncAnimation(fig, drawframe, frames=N+1, interval=20/animx, blit=True)
    return(anim)

## 1. Translation
$$
\begin{bmatrix}
    x_1 \\ x_2 \\ x_3
\end{bmatrix}
=
\begin{bmatrix}
    X_1 \\ X_2 \\ X_3
\end{bmatrix}
+
\begin{bmatrix}
    t \\ t \\ 0
\end{bmatrix}
\implies
\begin{align*}
    x_1 &= X_1+t\\
    x_2 &= X_2+t\\
    x_3 &= X_3
\end{align*}
$$

In [63]:
x1 = 'X1+t'
x2 = 'X2+t'
x3 = 'X3'
postproc(x1, x2, x3, T=10, N=30, P = [1,1,1], limit=[-2, 12, -2, 12], animx=1)


0.0 3.7007434154171886e-16 0.0 3.7007434154171886e-16 0.0 3.700743419208433e-16 0.0 3.70074341507253e-16
0 0 0 0


## 2. Rotation
$$
\begin{bmatrix}
    x_1 \\ x_2 \\ x_3
\end{bmatrix}
=
\begin{bmatrix}
    \cos\omega t & -\sin\omega t & 0 \\
    \sin\omega t & \cos\omega t & 0 \\
    0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
    X_1 \\ X_2 \\ X_3
\end{bmatrix}
\implies
\begin{align*}
    x_1 &= X_1\cos\omega t - X_2\sin\omega t \\
    x_2 &= X_1\sin\omega t + X_2\cos\omega t \\
    x_3 &= X_3
\end{align*}
$$

$$
\omega t = 90^o \implies \omega = 9^o
$$

In [64]:
x1 = 'X1*math.cos(math.radians(9*t))-X2*math.sin(math.radians(9*t))'
x2 = 'X1*math.sin(math.radians(9*t))+X2*math.cos(math.radians(9*t))'
x3 = 'X3'
postproc(x1, x2, x3, T=10, N=30, P = [1,1,1], limit=[-7, 7, -7, 7], animx=1.5)

0.0 9.344802340137745e-17 0.0 9.344802340137745e-17 0.0 9.436830774653943e-17 0.0 9.436830776046316e-17
0 0 0 0


/usr/lib/python3/dist-packages/matplotlib/animation.py:889: UserWarning: Animation was deleted without rendering anything. This is most likely not intended. To prevent deletion, assign the Animation to a variable, e.g. `anim`, that exists until you have outputted the Animation using `plt.show()` or `anim.save()`.
  warnings.warn(


## 3. Translation + Rotation
$$
\begin{bmatrix}
    x_1 \\ x_2 \\ x_3
\end{bmatrix}
=
\begin{bmatrix}
    \cos\omega t & -\sin\omega t & 0 \\
    \sin\omega t & \cos\omega t & 0 \\
    0 & 0 & 1
\end{bmatrix}
\begin{bmatrix}
    X_1 \\ X_2 \\ X_3
\end{bmatrix}
+
\begin{bmatrix}
    t \\ t \\ 0
\end{bmatrix}
\implies
\begin{align*}
    x_1 &= X_1\cos\omega t - X_2\sin\omega t + t\\
    x_2 &= X_1\sin\omega t + X_2\cos\omega t + t\\
    x_3 &= X_3
\end{align*}
$$
$$
\omega t = 90^o \implies \omega = 9^o
$$

In [65]:
x1 = 'X1*math.cos(math.radians(9*t))-X2*math.sin(math.radians(9*t))+t'
x2 = 'X1*math.sin(math.radians(9*t))+X2*math.cos(math.radians(9*t))+t'
x3 = 'X3'
postproc(x1, x2, x3, T=10, N=30, P = [1,1,1], limit=[-2, 12, -2, 12], animx=1)

0.0 5.551150995535655e-16 0.0 5.551150995535655e-16 0.0 5.551186878053594e-16 0.0 5.551186867197926e-16
0 0 0 0


## 4. Pure Shear
$$
\begin{align*}
    x_1 &= X_1 + X_2t\\
    x_2 &= X_2\\
    x_3 &= X_3
\end{align*}
$$

In [66]:
x1 = 'X1+X2*t'
x2 = 'X2'
x3 = 'X3'
postproc(x1, x2, x3, T=10, N=300, P = [1,1,1], limit=[-7, 7, -7, 7], animx=1)

0.0 1482.8706774295883 0.0 1424.337653151856 0.0 42.00970303478699 0.0 42.009702885538346
0 0 1483 42


## 5. Generic
$$
\begin{align*}
    x_1 &= X_1e^t + X_3(e^t-1)\\
    x_2 &= X_2 + X_3(e^t-e^{-t})\\
    x_3 &= X_3
\end{align*}
$$

In [67]:
x1 = 'X1*math.exp(t)+X3*(math.exp(t)-1)'
x2 = 'X2+X3*(math.exp(t)-math.exp(-t))'
x3 = 'X3'
postproc(x1, x2, x3, T=2, N=30, P = [0,0,0], limit=[-7, 7, -7, 7], animx=0.5)

0.0 299.536419207156 0.0 19.994482236454377 0.0 52062311.87220356 0.0 6.052191281275271
0 0 300 52062312


## 6. Biaxial Tension
$$
\begin{align*}
    x_1 &= X_1 + X_1t\\
    x_2 &= X_2 + X_2t\\
    x_3 &= X_3
\end{align*}
$$

In [68]:
x1 = 'X1+X1*t'
x2 = 'X2+X2*t'
x3 = 'X3'
postproc(x1, x2, x3, T=10, N=30, P = [1,1,1], limit=[-7, 7, -7, 7], animx=1)

0.0 46.99074074074073 0.0 0.4130045572916667 0.0 2607020.494486319 0.0 0.41300455728924135
0 0 47 2607020


## 7. Biaxial Compression
$$
\begin{align*}
    x_1 &= X_1 - X_1t\\
    x_2 &= X_2 - X_2t\\
    x_3 &= X_3
\end{align*}
$$

In [69]:
x1 = 'X1-X1*t'
x2 = 'X2-X2*t'
x3 = 'X3'
postproc(x1, x2, x3, T=1, N=30, P = [1,1,1], limit=[-7, 7, -7, 7], animx=1.2)

0.0 0.41620370370370374 0.0 374.5833333333335 0.0 3397418648.0436435 0.0 374.58333349227905
0 0 0 3397418648


## 8. Plane Strain Compression
$$
\begin{align*}
    x_1 &= X_1 + X_1t\\
    x_2 &= X_2 - X_2t\\
    x_3 &= X_3
\end{align*}
$$

In [70]:
x1 = 'X1+X1*t'
x2 = 'X2-X2*t'
x3 = 'X3'
postproc(x1, x2, x3, T=1, N=30, P = [1,1,1], limit=[-7, 7, -7, 7], animx=1.2)

0.0 2.0869002900581335 0.0 118990.85129183433 0.0 1361476781.628001 0.0 22.096825660369383
0 0 2 1361476782
